# ReAct Bot

In [1]:
#| default_exp react

In [2]:
#| export
import re

from fastcore.all import *

from chat_react.core import *

## Exceptions

In [3]:
#| export
class CancelCallTool(Exception): pass
class CancelStep(Exception): pass
class CancelLLM(Exception): pass
class CancelParse(Exception): pass
class CancelRun(Exception): pass

## Callbacks

In [38]:
#| export
class LogMessagesCB:
    def after_llm(self, bot, out): print(out[0])
    def after_call_tool(self, bot, out): print(f'Output: {out}')

In [39]:
#| export
class CostCB:
    def __init__(self, cost=0): store_attr()
    def after_llm(self, bot, args):
        print(f'Cost: ${args[1]}')
        self.cost += args[1]
    def after_run(self, bot, cost):
        print(f'Total cost: ${self.cost}')

In [40]:
#| export
class React:
    def __init__(self, prompt=None, examples=[], tools=[], cbs=[], max_turns=10, end_tool='finish'):
        store_attr()
        if not prompt:
            prompt = self.base_prompt(tools)
        self.messages = [hm(prompt)] + examples + [aim("Though: I have to greet the user\nAction: send_to_user[Hi, I am an AI, how can I help you today?]")]
    
    def base_prompt(self, tools): 
        prompt = "You can use the following tools:\n" + '\n'.join([f"- {tool.__name__}" for tool in tools])
        prompt += "Before using a tool think about what you want to do and write it down in a sentence.\n"
        prompt += "Use the following format:\nThougth: <your thought>\nAction: <tool_name>[<tool_input>]\n\n"
        return prompt
    
    def llm(self, messages): return chat_create(messages)

    def with_cbs(self, name, exception, *args, **kwargs):
        for cb in self.cbs:
            if hasattr(cb, f'before_{name}'): getattr(cb, f'before_{name}')(self, *args, **kwargs)
        try: result = getattr(self, name)(*args, **kwargs) 
        except exception as e:
            for cb in self.cbs:
                if hasattr(cb, f'after_cancel_{name}'): getattr(cb, f'after_cancel_{name}')(self, *args, **kwargs)
            return
        for cb in self.cbs:
            if hasattr(cb, f'after_{name}'): getattr(cb, f'after_{name}')(self, result)
        return result
    
    def __call__(self, query):
        return self.with_cbs('run', CancelRun, self.messages + [hm(f'Output: {query}')])
    
    def run(self, messages):
        for i in range(self.max_turns):
            messages, tool_name, tool_input = self.with_cbs('step', CancelStep, messages)
            if tool_name == self.end_tool: break
        return messages, tool_name, tool_input


    def step(self, messages):
        out, cost = self.with_cbs('llm', CancelLLM, messages)
        messages.append(aim(out))
        tool_name, tool_input = self.with_cbs('parse_output', CancelParse, out)
        if tool_name != self.end_tool:
            tool_output = self.with_cbs('call_tool', CancelCallTool, tool_name, tool_input)
            messages.append(hm(f"Output: {tool_output}"))
        return messages, tool_name, tool_input
    
    def parse_output(self, text):
        """Parses text into action and action_input."""
        i = text.find('Action:')
        m = re.match(r"Action: (?P<tool_name>.*)\[(?P<tool_input>.*)\]", text[i:], re.DOTALL)
        if not m: raise Exception
        return m.group(1, 2)

    def call_tool(self, tool_name, tool_input):
        for tool in self.tools: 
            if tool.__name__ == tool_name: return tool(tool_input)

In [41]:
#| export
def calculator(s):
    """python interpreter"""
    return eval(s)

In [42]:
#| export
def send_to_user(m):
    """ask a question to the user"""
    print(m)
    return input

In [43]:
#| export
def finish():
    """send your final answer to the user"""

In [44]:
examples = [aim("Though: I have to greet the user\nAction: send_to_user[Hi, I am an AI, how can I help you today?]"),
            hm("Output: how much is 45+32"),
            aim("Though: I have to calculate 45+32\nAction: calculator[45+32]"),
            hm("Output: 77"),aim("Though: I know the answer to the user question\nAction: finish[77]")]

In [45]:
bot = React(examples=examples, tools=[calculator, send_to_user, finish], cbs=[LogMessagesCB(), CostCB()])

In [46]:
x = 1951
y = 95109
_, _, out = bot(f"how much is {x}*{y}")
test_eq(int(out), x*y)

Though: I have to calculate 1951*95109
Action: calculator[1951*95109]
Cost: $0.000488
Output: 185557659
Though: I know the answer to the user question
Action: finish[185557659]
Cost: $0.000558
Total cost: $0.001046


In [47]:
out, cost = chat_create([hm(f"how much is {x}*{y}. output only the result with no commas")])
print(f'Total cost: ${cost}')
test_ne(int(out), x*y)

Total cost: $5.6e-05


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()